In [105]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [106]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"UNFI"}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':112.486}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':65.815}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':22.622}, 
             {'symbol':"PYPL", 'basis':69.910}, 
             {'symbol':"ALL"}, 
             {'symbol':"LC", 'basis':9.900}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':54.610}, 
             {'symbol':"CCL", 'basis':16.147}, 
             {'symbol':"AGX", 'basis':41.148}, 
             {'symbol':"POWL", 'basis':187.000}, 
             {'symbol':"PPC", 'basis':45.054}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':78.359}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA"}, 
             {'symbol':"TWLO", 'basis':98.403}, 
             {'symbol':"APP", 'basis':248.047}, 
             {'symbol':"TMUS"}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':50.777}, 
             {'symbol':"CAAP", 'basis':21.57}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':86.654}, 
                   {'symbol':"ALL"}, 
                   {'symbol':"APP", 'basis':225.605}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"BRK-B"}, 
                   {'symbol':"CCL", 'basis':20.112}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"COMM"}, 
                   {'symbol':"CRDO", 'basis':38.629}, 
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"EZPW", 'basis':18.176}, 
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.19}, 
                   {'symbol':"NVTS"}, 
                   {'symbol':"OKTA", 'basis':106.18}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"STRL"}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"UBER"}, 
                   {'symbol':"WFC", 'basis':68.586}
                   ]

In [107]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [108]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=1, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [109]:
brokerage_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,86.654,207.02,141.24,72.64,150.88,34.26,8.130,0.0071,Sell
1,ALL,0.000,206.75,196.68,194.49,202.40,14.16,2.737,0.0193,Sell
2,APP,225.605,384.23,278.87,17.91,305.42,70.44,224.000,0.0000,Sell
3,ARQT,15.996,13.67,13.96,13.96,15.25,inf,11.820,0.0000,Hold
4,ATGE,86.385,129.80,107.12,89.00,112.38,21.93,3.326,0.0000,Sell
5,BRK-B,0.000,506.31,507.46,628.82,520.96,13.41,1.658,0.0000,Hold
6,CCL,20.112,23.16,19.39,18.87,20.42,15.23,3.393,0.0000,Sell
7,CLS,74.924,116.43,86.42,40.09,93.25,33.18,8.720,0.0000,Sell
8,COMM,0.000,6.14,4.69,4.69,5.11,inf,30.360,0.0000,Sell
9,CRDO,38.629,63.83,44.20,1.97,48.18,2090.33,17.220,0.0000,Sell


In [110]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=126)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [111]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,EAT,112.486,168.56,136.51,38.70,151.41,22.98,28.310,0.0000,Sell
3,EZPW,15.460,13.42,13.50,24.20,14.74,11.21,0.941,0.0000,Hold
4,ARQT,15.932,13.67,12.70,12.70,14.59,inf,11.820,0.0000,Hold
5,WFC,65.815,74.53,67.51,98.62,73.19,13.26,1.477,0.0217,Sell
6,ITRN,35.710,37.26,33.90,28.98,37.77,14.54,3.899,0.0510,Hold
7,CRDO,22.622,63.83,52.19,1.97,60.15,2090.33,17.220,0.0000,Sell
8,PYPL,69.910,70.93,69.90,55.98,76.06,16.05,3.430,0.0000,Hold
10,LC,9.900,10.14,12.21,13.50,13.59,22.91,0.844,0.0000,Hold
12,CLS,54.610,116.43,90.55,40.09,103.11,33.18,8.720,0.0000,Sell
13,CCL,16.147,23.16,20.76,18.87,22.87,15.23,3.393,0.0000,Sell


In [112]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,UNFI,0.0,30.96,25.57,25.57,28.33,inf,1.133,0.0000,Sell
1,MFC,0.0,31.74,28.42,35.79,30.72,16.41,1.643,0.0403,Sell
9,ALL,0.0,206.75,183.67,194.49,198.65,14.16,2.737,0.0193,Sell
11,QTWO,0.0,86.72,82.38,82.38,90.39,inf,10.120,0.0000,Hold
17,SYF,0.0,58.25,54.29,99.16,59.44,8.09,1.463,0.0203,Hold
18,ATGE,0.0,129.80,96.98,89.00,106.41,21.93,3.326,0.0000,Sell
19,BRK-B,0.0,503.11,464.16,624.84,499.62,13.41,1.658,0.0000,Sell
20,SFM,0.0,167.43,141.54,45.60,155.51,37.04,12.480,0.0000,Sell
22,BLBD,0.0,39.17,34.77,25.59,38.58,12.14,6.620,0.0000,Sell
23,GM,0.0,48.59,45.70,128.21,49.88,6.72,0.733,0.0125,Hold


In [113]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'} # Mexico - Communications - mobile network
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [114]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,24.56,23.07,27.74,23.67,10.90,2.466,0.0553,Sell
1,KARO,0,55.50,43.44,18.49,46.24,32.65,9.460,0.0198,Sell
2,QFIN,0,42.44,40.69,72.27,43.09,6.60,1.792,0.0305,Hold
3,TSM,0,197.19,165.54,92.20,172.32,23.52,6.670,0.0168,Sell
4,CCU,0,13.87,14.51,17.03,14.98,14.45,1.575,0.0329,Hold
5,ABEV,0,2.48,2.34,2.42,2.40,15.50,2.325,0.0609,Sell
6,TIMB,0,17.16,15.37,19.45,15.73,13.75,1.942,0.0892,Sell
7,ASR,0,318.18,292.33,273.25,302.85,13.64,3.409,0.0768,Sell
8,RYAAY,0,56.62,47.22,41.45,48.53,17.05,3.753,0.0166,Sell
9,AMX,0,17.32,15.50,11.16,15.93,36.23,2.280,0.0291,Sell


In [115]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [116]:
mag7_test_table

[['MSFT', 0, 458.68, 354.56, 138.56, 404.83, 35.59, 10.56, 0.0072, 'Sell'],
 ['AAPL', 0, 199.95, 181.97, 31.28, 210.67, 31.27, 44.81, 0.0052, 'Hold'],
 ['NVDA', 0, 139.19, 99.82, 18.7, 116.92, 45.85, 41.44, 0.0003, 'Sell'],
 ['AMZN', 0, 205.7, 168.52, 78.24, 194.93, 33.34, 7.11, 0, 'Sell'],
 ['GOOG', 0, 172.96, 143.76, 93.15, 165.66, 19.35, 6.11, 0.0048, 'Sell'],
 ['GOOGL', 0, 171.86, 142.03, 93.21, 163.75, 19.24, 6.06, 0.0049, 'Sell'],
 ['META', 0, 645.05, 510.91, 254.73, 592.44, 25.19, 8.73, 0.0033, 'Sell'],
 ['BRK-B', 0, 503.11, 456.63, 624.84, 520.9, 13.41, 1.658, 0, 'Hold'],
 ['TSLA', 0, 358.43, 238.04, 37.14, 282.82, 203.94, 15.66, 0, 'Sell']]

In [117]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,458.68,354.56,138.56,404.83,35.59,10.560,0.0072,Sell
1,AAPL,0,199.95,181.97,31.28,210.67,31.27,44.810,0.0052,Hold
2,NVDA,0,139.19,99.82,18.70,116.92,45.85,41.440,0.0003,Sell
3,AMZN,0,205.70,168.52,78.24,194.93,33.34,7.110,0.0000,Sell
4,GOOG,0,172.96,143.76,93.15,165.66,19.35,6.110,0.0048,Sell
5,GOOGL,0,171.86,142.03,93.21,163.75,19.24,6.060,0.0049,Sell
6,META,0,645.05,510.91,254.73,592.44,25.19,8.730,0.0033,Sell
7,BRK-B,0,503.11,456.63,624.84,520.90,13.41,1.658,0.0000,Hold
8,TSLA,0,358.43,238.04,37.14,282.82,203.94,15.660,0.0000,Sell


In [118]:
# Machine Learning

In [119]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [120]:
# Feature selection and normalization

In [121]:
# Build model

In [122]:
# Train model

In [123]:
# Evaluate model

In [124]:
# Refine model through hyperparameter tuning